In [ ]:
# use "%" operator (jupyter magic commands) as a partial shell 

In [4]:
#%pip install duckdb
import duckdb

In [5]:
%pwd

'/Users/aramji/dev/github/meds/eds213/bren-meds213-spring-2024-class-data/week3'

Create connection and a cursor

In [6]:
conn = duckdb.connect("database.db")

In [7]:
conn

In [8]:
cur = conn.cursor()

Now let's do something with our cursor

In [10]:
cur.execute("SELECT * FROM Site LIMIT 5")

Now we want results... 3 ways of getting them.

1. All results at once

In [12]:
cur.fetchall()

[('barr',
  'Barrow',
  'Alaska, USA',
  71.30000305175781,
  -156.60000610351562,
  220.39999389648438),
 ('burn',
  'Burntpoint Creek',
  'Ontario, Canada',
  55.20000076293945,
  -84.30000305175781,
  63.0),
 ('bylo',
  'Bylot Island',
  'Nunavut, Canada',
  73.19999694824219,
  -80.0,
  723.5999755859375),
 ('cakr',
  'Cape Krusenstern',
  'Alaska, USA',
  67.0999984741211,
  -163.5,
  54.099998474121094),
 ('cari',
  'Canning River Delta',
  'Alaska, USA',
  70.0999984741211,
  -145.8000030517578,
  722.0)]

you can also have it returned as a pd.dataframe (show later)

Cursors don't store anything, they just transfer queries to the database and get results back.

In [13]:
cur.fetchall()

[]

Cursor doesn't remember your list of 5 tuples... you'd have to rerun the cur.execute("query") and then run fetchall() again. Up to you to save the results if you want to save them 


Always get tuples, even if you only request one column.

In [14]:
cur.execute("SELECT Nest_ID FROM Bird_nests LIMIT 10")

In [15]:
cur.fetchall()

[('14HPE1',),
 ('11eaba',),
 ('11eabaagc01',),
 ('11eabaagv01',),
 ('11eababbc02',),
 ('11eababsv01',),
 ('11eabaduh01',),
 ('11eabaduv01',),
 ('11eabarpc01',),
 ('11eabarpc02',)]

even though you've only requested 1 column, you still get back a tuple... a 1 tuple (value,).

In [16]:
cur.execute("SELECT Nest_ID FROM Bird_nests LIMIT 10")
[t[0] for t in cur.fetchall()]

['14HPE1',
 '11eaba',
 '11eabaagc01',
 '11eabaagv01',
 '11eababbc02',
 '11eababsv01',
 '11eabaduh01',
 '11eabaduv01',
 '11eabarpc01',
 '11eabarpc02']

list comprehension

compact & useful  -- mini loop inside the square brackets

give first element from each returned entry in the previous command

2. Get the one result, or the next result

In [18]:
cur.execute("SELECT COUNT(*) FROM Bird_nests")
cur.fetchall()

[(1547,)]

if you only want 1 row, use `fetchone()`

In [21]:
cur.execute("SELECT COUNT(*) FROM Bird_nests")
cur.fetchone()

(1547,)

In [22]:
cur.execute("SELECT COUNT(*) FROM Bird_nests")
cur.fetchone()[0]

1547

BTW, if you want to get help on things in Python, ask for help on a variable, will get back help on that variable's type.

In [24]:
help(cur)

Help on DuckDBPyConnection in module duckdb.duckdb object:

class DuckDBPyConnection(pybind11_builtins.pybind11_object)
 |  Method resolution order:
 |      DuckDBPyConnection
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __enter__(...)
 |      __enter__(self: duckdb.duckdb.DuckDBPyConnection) -> duckdb.duckdb.DuckDBPyConnection
 |  
 |  __exit__(...)
 |      __exit__(self: duckdb.duckdb.DuckDBPyConnection, exc_type: object, exc: object, traceback: object) -> None
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  append(...)
 |      append(self: duckdb.duckdb.DuckDBPyConnection, table_name: str, df: pandas.DataFrame, *, by_name: bool = False) -> duckdb.duckdb.DuckDBPyConnection
 |      
 |      Append the passed DataFrame to the named table
 |  
 |  array_type(...)
 |      array_type(self: duckdb.duckdb.DuckDBPyConnection, type: duckdb.duckdb.typing.Duck

3. Using an iterator - but DuckDB doesn't support iterators :(

In [25]:
cur.execute("SELECT Nest_ID FROM Bird_nests LIMIT 10")
for row in cur:
    print(f"got {row[0]}")

TypeError: 'duckdb.duckdb.DuckDBPyConnection' object is not iterable

for some reason, duckdb doesn't... every other db usually does.

A workaround:

Can use cursor as iterator.  Technique for large returns, more efficient than `fetchall()`.

In [26]:
cur.execute("SELECT Nest_ID FROM Bird_nests LIMIT 10")
while True:
    row = cur.fetchone()
    if row == None:
        break
    # do something with row
    print(f"got nest ID {row[0]}")

got nest ID 14HPE1
got nest ID 11eaba
got nest ID 11eabaagc01
got nest ID 11eabaagv01
got nest ID 11eababbc02
got nest ID 11eababsv01
got nest ID 11eabaduh01
got nest ID 11eabaduv01
got nest ID 11eabarpc01
got nest ID 11eabarpc02


especially useful for bigger data.

if not using duckdb, you can use the first for loop

Can do things other than SELECT!

use """, otherwise u get unterminated string literal SyntaxError

In [27]:
# """ is way of quoting a multilinestring
cur.execute(""" 
    CREATE TEMP TABLE temp_table AS
    SELECT * FROM Bird_nests LIMIT 10
    """ )

In [28]:
cur.execute("SELECT * from temp_table")

In [29]:
cur.fetchall()

[('b14.6',
  2014,
  'chur',
  '14HPE1',
  'sepl',
  'vloverti',
  datetime.date(2014, 6, 14),
  None,
  3,
  None,
  None),
 ('b11.7',
  2011,
  'eaba',
  '11eaba',
  'wrsa',
  'bhill',
  datetime.date(2011, 7, 10),
  'searcher',
  4,
  None,
  None),
 ('b11.6',
  2011,
  'eaba',
  '11eabaagc01',
  'amgp',
  'dkessler',
  datetime.date(2011, 6, 24),
  'searcher',
  4,
  6.0,
  'float'),
 ('b11.6',
  2011,
  'eaba',
  '11eabaagv01',
  'amgp',
  'dkessler',
  datetime.date(2011, 6, 25),
  'searcher',
  3,
  3.0,
  'float'),
 ('b11.6',
  2011,
  'eaba',
  '11eababbc02',
  'bbpl',
  'dkessler',
  datetime.date(2011, 6, 24),
  'searcher',
  4,
  4.0,
  'float'),
 ('b11.7',
  2011,
  'eaba',
  '11eababsv01',
  'wrsa',
  'bhill',
  datetime.date(2011, 7, 7),
  'searcher',
  4,
  2.0,
  'float'),
 ('b11.6',
  2011,
  'eaba',
  '11eabaduh01',
  'dunl',
  'dkessler',
  datetime.date(2011, 6, 28),
  'searcher',
  3,
  2.0,
  'float'),
 ('b11.6',
  2011,
  'eaba',
  '11eabaduv01',
  'dunl',
  'dk

A note on fragility

For example:
one way to do an insert statement: order of values has to align with order of the columns
```{SQL}
INSERT INTO Site VALUES ("abcd", "Foo", 35.7, -119.5, "?") 
```
not necessarily obvious which value is supposed to go with which column or if code was correct

a less fragile way of expressing the same thing:
be explicit
```{SQL}
INSERT INTO Site (Code, Site_name, Latitude, Longitude, Something_else)
    VALUES ("abcd", "Foo", 35.7, -119.5, "?")
```

In the same vein: SELECT * is fragile

In [30]:
cur.execute("SELECT * FROM Site LIMIT 3")
cur.fetchall()

[('barr',
  'Barrow',
  'Alaska, USA',
  71.30000305175781,
  -156.60000610351562,
  220.39999389648438),
 ('burn',
  'Burntpoint Creek',
  'Ontario, Canada',
  55.20000076293945,
  -84.30000305175781,
  63.0),
 ('bylo',
  'Bylot Island',
  'Nunavut, Canada',
  73.19999694824219,
  -80.0,
  723.5999755859375)]

just tuples of values, nothing that tells us which value goes to which column, what those columns are

a better, more robust way of coding the same thing:

In [31]:
cur.execute("SELECT Site_name, Code, Latitude, Longitude FROM Site LIMIT 3")
cur.fetchall()

[('Barrow', 'barr', 71.30000305175781, -156.60000610351562),
 ('Burntpoint Creek', 'burn', 55.20000076293945, -84.30000305175781),
 ('Bylot Island', 'bylo', 73.19999694824219, -80.0)]

when you're programming, don't use SELECT * -- try to be explicit.



How many bird nests do we have for each species? (could do in 1 query, but we're going to use a more programmatic approach as an example)

Approach: first get all species. Then execute a count query for each species.

execute query once for each species ('%')

In [32]:
query = """
    SELECT COUNT(*) FROM Bird_nests
    WHERE Species = '%s'
"""

cur.execute("SELECT Code FROM Species LIMIT 3")

for row in cur.fetchall(): # DuckDB workaround
    code = row[0]
    prepared_query = query % code # interpolate code inside the query
    print(prepared_query)


    SELECT COUNT(*) FROM Bird_nests
    WHERE Species = 'agsq'


    SELECT COUNT(*) FROM Bird_nests
    WHERE Species = 'amcr'


    SELECT COUNT(*) FROM Bird_nests
    WHERE Species = 'amgp'



A digression: string interpolation in Python

In [39]:
# The % method
s = "My name is %s"
print(s % "Greg")
s = "My name is %s and the other teacher's name is %s" # can use multiple
print(s % ("Greg", "Julien"))

# the new f-string method
name = "Greg"
print(f"My name is {name}")

# Third way
print("My name is {}".format("Greg"))

My name is Greg
My name is Greg and the other teacher's name is Julien
My name is Greg
My name is Greg


%s is a placeholder code

% operator 

My name is Greg and the other teacher's name is Julien


In [44]:
query = """
    SELECT COUNT(*) FROM Bird_nests
    WHERE Species = '%s'
"""

cur.execute("SELECT Code FROM Species LIMIT 3")

for row in cur.fetchall(): # DuckDB workaround
    code = row[0]
    prepared_query = query % code # interpolate code inside the query
    #print(prepared_query)
    cur2 = conn.cursor() # case-sensitive
    cur2.execute(prepared_query)
    print(f"Species {code} has {cur2.fetchone()[0]} nests")
    cur2.close() # for good form, close that cursor

Species agsq has 0 nests
Species amcr has 0 nests
Species amgp has 29 nests


have to close that connection if you want to access your database from somewhere else -- can't open it as long as that kernel and connection are running and alive.

The above Python interpolation is actually rather dangerous. It has caused many database hacks!
and there's a better way to do it... !



swap '%' with ? (no quotes)

In [45]:
query = """
    SELECT COUNT(*) FROM Bird_nests
    WHERE Species = ?
"""

cur.execute("SELECT Code FROM Species LIMIT 3")

for row in cur.fetchall(): # DuckDB workaround
    code = row[0]
    #NOT NEEEDED! prepared_query = query % code
    #print(prepared_query)
    cur2 = conn.cursor() # case-sensitive
    cur2.execute(query, [code]) # <-- added argument here
    print(f"Species {code} has {cur2.fetchone()[0]} nests")
    cur2.close() # for good form, close that cursor

Species agsq has 0 nests
Species amcr has 0 nests
Species amgp has 29 nests


Rather than doing string manipulation, better to use those extra parameters 


Let's illustrate the danger with a different example

In [46]:
abbrev = "TS"
name = "Taylor Swift"
cur.execute("""
        INSERT INTO Personnel (Abbreviation, Name)
            VALUES ('%s', '%s')    
        """ % (abbrev, name)
            )

In [47]:
cur.execute("SELECT * FROM Personnel")
cur.fetchall()[-3:]

[('emagnuson', 'Emily Magnuson'),
 ('mcorrell', 'Maureen Correll'),
 ('TS', 'Taylor Swift')]

used Python interpolation, '%s' 

In [48]:
abbrev = "CO"
name = "Conan O'Brien"
cur.execute("""
        INSERT INTO Personnel (Abbreviation, Name)
            VALUES ('%s', '%s')    
        """ % (abbrev, name)
            )

ParserException: Parser Error: syntax error at or near "Brien"

In [50]:
"""
        INSERT INTO Personnel (Abbreviation, Name)
            VALUES ('%s', '%s')    
        """ % (abbrev, name)
            

"\n        INSERT INTO Personnel (Abbreviation, Name)\n            VALUES ('CO', 'Conan O'Brien')    \n        "

issue bc ' in name

In [53]:
abbrev = "CO"
name = "Conan O'Brien"
cur.execute("""
        INSERT INTO Personnel (Abbreviation, Name)
            VALUES (?, ?)    
        """,  # use ? and [variable, variable] instead of string
[abbrev, name] # updated argument here
)

In [54]:
cur.execute("SELECT * FROM Personnel")
cur.fetchall()[-3:]

[('mcorrell', 'Maureen Correll'),
 ('TS', 'Taylor Swift'),
 ('CO', "Conan O'Brien")]